In [ ]:
# !pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.2 MB/s eta 0:00:00


In [ ]:
# !pip install asyncpg

   ---------------------------------------- 0.0/625.8 kB ? eta -:--:--
   ---------------------------------------- 625.8/625.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
# !pip install passlib

   ---------------------------------------- 0.0/525.6 kB ? eta -:--:--
   ---------------------------------------  524.3/525.6 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 525.6/525.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
# !pip install bcrypt

  Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl.metadata (10 kB)
Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl (153 kB)


In [2]:
import passlib

In [13]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="sql_project_p3",
    user="postgres",
    password="admin123"
)


In [15]:
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a sample query
cursor.execute("SELECT emp_id, user_name, password_hash FROM employees;")

# Fetch all rows
rows = cursor.fetchall()

# Print data
for row in rows:
    print(row)

# Close connection
cursor.close()
conn.close()

InterfaceError: connection already closed

In [1]:
from passlib.context import CryptContext
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="sql_project_p3",
    user="postgres",
    password="admin123"
)
# Setup password hashing
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

cursor = conn.cursor()

# Employee credentials
employees = [
    ('E101', 'johe101', 'joh101'),
    ('E102', 'jane102', 'jan102'),
    ('E103', 'mike103', 'mik103'),
    ('E104', 'emie104', 'emi104'),
    ('E105', 'sare105', 'sar105'),
    ('E106', 'mice106', 'mic106'),
    ('E107', 'mice107', 'mic107'),
    ('E108', 'jese108', 'jes108'),
    ('E109', 'dane109', 'dan109'),
    ('E110', 'laue110', 'lau110'),
    ('E111', 'chre111', 'chr111')
]

# Insert hashed passwords
for emp_id, username, password in employees:
    hashed_password = pwd_context.hash(password)
    cursor.execute("UPDATE employees SET user_name = %s, password_hash = %s WHERE emp_id = %s", 
                   (username, hashed_password, emp_id))

conn.commit()
cursor.close()
conn.close()


(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "c:\Users\kmnik\anaconda3\envs\env_auc\lib\site-packages\passlib\handlers\bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'


In [11]:
import psycopg2
from passlib.context import CryptContext

# Initialize bcrypt password hashing
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def verify_employee_login(username, entered_password):
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
        host="localhost",
        database="sql_project_p3",
        user="postgres",
        password="admin123"
        )
        cursor = conn.cursor()

        # Fetch employee record by username
        cursor.execute("SELECT password_hash FROM employees WHERE user_name = %s", (username,))
        user_record = cursor.fetchone()
        print(user_record)
        # Close database connection
        cursor.close()
        conn.close()

        # Check if user exists
        if not user_record:
            return "User not found"

        stored_hash = user_record[0]  # Get the password hash from the tuple

        # Verify password
        if pwd_context.verify(entered_password, stored_hash):
            return "Login successful"
        else:
            return "Invalid credentials"

    except Exception as e:
        return f"Database error: {str(e)}"

# Test login
print(verify_employee_login("johe101", "joh101"))  # ✅ Should return "Login successful"
print(verify_employee_login("jane102", "wrongpass"))    # ❌ Should return "Invalid credentials"


('$2b$12$udJLDgoO68L5mRKRH6M8..sigttGp9zTJ7HzFvZHzS/21JKncYsM.',)
Login successful
('$2b$12$aGisPtlIHAhqhanmGcJ.J.aJhGgaao5cVvlAPzKEk0gxzc3YngGH6',)
Invalid credentials


In [7]:

import psycopg2
DB_CONFIG = {
    "host" : "localhost",
    "database" : "sql_project_p3",
    "user" : "postgres",
    "password" : "admin123"
}

def get_db_connection():
    """Establish connection to PostgreSQL"""
    return psycopg2.connect(**DB_CONFIG)
# class IssueBookRequest(BaseModel):
#     issued_id: str
#     member_id: str
#     book_isbn: str
#     employee_id: str

def issue_book(issued_id,member_id,book_isbn, employee_id):
    """Issue a book by calling stored procedure add_issue_book()"""
# try:
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # cursor.execute("CALL add_issue_book(%s, %s, %s, %s)", 
    #                 (issued_id,member_id,book_isbn, employee_id))
    # Fetch book title
    cursor.execute("SELECT book_title FROM books WHERE isbn = %s", 
                (book_isbn,))  
    book_title = cursor.fetchone()
    print(book_title[0])
    conn.commit()

    cursor.close()
    conn.close()
    
    return {"message": f"The Book **{book_title[0]}** issued successfully"}
    
    # except Exception as e:
    #     raise HTTPException(status_code=500, detail=str(e))


In [8]:
issue_book(
"IS146",
"C106",
"978-0-14-118776-1",
"E104"
)

One Hundred Years of Solitude


{'message': 'The Book **One Hundred Years of Solitude** issued successfully'}

In [15]:
def return_book(return_id, issued_id):
    """Return a book by calling stored procedure add_return_status()"""
    # try:
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("CALL add_return_status(%s, %s)", 
                    (return_id, issued_id))
    
    conn.commit()
    cursor.close()
    conn.close()
    
    return {"message": "Book returned successfully"}

In [16]:
return_book('RS141','IS141')

{'message': 'Book returned successfully'}

In [41]:
def get_book_status(book_isbn: str):
    """Check if a book is available or not"""
# try:
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("SELECT status,book_title FROM books WHERE isbn = %s", (book_isbn,))
    book_status = cursor.fetchone()
    print(book_status)
    cursor.close()
    conn.close()
    
    if book_status[0] == 'yes':
        # print('AVAILABLE')
        return {"status:":[f"**The Book**",book_status[1],'**available**']}  # 'available' or 'not available'
    elif book_status[0] == 'no':
        return {"status": "Book is not Available"} 

In [42]:
get_book_status("978-0-679-76489-8")

('yes', 'Harry Potter and the Sorcerers Stone')


{'status:': ['**The Book**',
  'Harry Potter and the Sorcerers Stone',
  '**available**']}

In [ ]:
import streamlit as st
import psycopg2
from passlib.context import CryptContext
import requests

# FastAPI backend URL
API_URL = "http://127.0.0.1:8000"

# Streamlit UI
st.title("📚 Library Management System")

# Select an action
action = st.selectbox("Select Action", ["Issue a Book", "Return a Book", "Check Book Status"])

if action == "Issue a Book":
    st.subheader("📖 Issue a Book")
    issued_id = st.text_input("Issued ID")
    member_id = st.text_input("Member ID")
    book_isbn = st.text_input("Book ISBN")
    employee_id = st.text_input("Employee ID")

    if st.button("Issue Book"):
        payload = {
            "issued_id": issued_id,
            "member_id": member_id,
            "book_isbn": book_isbn,
            "employee_id": employee_id
        }
        response = requests.post(f"{API_URL}/issue_book", json=payload)
        st.success(response.json()["message"]) if response.status_code == 200 else st.error(response.json()["error"])

elif action == "Return a Book":
    st.subheader("🔄 Return a Book")
    return_id = st.text_input("Return ID")
    issued_id = st.text_input("Issued ID")

    if st.button("Return Book"):
        payload = {
            "return_id": return_id,
            "issued_id": issued_id
        }
        response = requests.post(f"{API_URL}/return_book", json=payload)
        st.success(response.json()["message"]) if response.status_code == 200 else st.error(response.json()["error"])

elif action == "Check Book Status":
    st.subheader("✅ Check Book Status")
    book_isbn = st.text_input("Enter Book ISBN")

    if st.button("Check Status"):
        response = requests.get(f"{API_URL}/book_status/{book_isbn}")
        if response.status_code == 200:
            st.info(f"Book Status: {response.json()['status']}")
        else:
            st.error(response.json()["error"])
